<a href="https://colab.research.google.com/github/ded-mikhalych/TK_SMM_LRs/blob/main/%D0%A1%D0%B0%D1%84%D0%BE%D0%BD%D0%BE%D0%B2%D0%B0_%D0%9F_%D0%94_%2C_%D0%9C%D0%B0%D0%BA%D1%81%D0%B8%D0%BC%D0%B5%D0%BD%D0%BA%D0%BE%D0%B2_%D0%9A_%D0%95_%2C_%D0%9C%D0%B8%D1%85%D0%B0%D0%BB%D1%8C%D1%87%D1%83%D0%BA_%D0%94_%D0%90__%D0%A2%D0%9A_%D0%9B%D0%A0_%E2%84%961.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random as rnd

In [ ]:
def ref(matrix):
    edited_matr = matrix.copy();

    row = edited_matr.shape[0]
    column = edited_matr.shape[1]

    lead_elem_idx = 0

    for j in range(column):
        non_zero_row_idxs = np.nonzero(edited_matr[ :, j] == 1)[0]
        if len(non_zero_row_idxs) > 0 and lead_elem_idx in non_zero_row_idxs:
            for i in range(lead_elem_idx + 1, row):
                if edited_matr[i][j] == 1:
                    edited_matr[i] = (edited_matr[i] + edited_matr[lead_elem_idx]) % 2
            lead_elem_idx += 1

    for i in range (row - 1, -1, -1):
        if np.any(edited_matr[i, :]) == 0:
            edited_matr = np.delete(edited_matr, i, 0)

    return edited_matr

In [ ]:
def rref(matrix):
    ref_matrix = ref(matrix)
    row = ref_matrix.shape[0]
    for i in range(row - 1, 0, -1):
        j = np.nonzero(ref_matrix[i] == 1)[0][0]
        for k in range(i - 1, -1, -1):
            if ref_matrix[k][j] == 1:
                ref_matrix[k] = (ref_matrix[k] + ref_matrix[i]) % 2
    return ref_matrix

In [ ]:
def reduce_matrix(matrix):
    edited_matr = rref(matrix)

    row = edited_matr.shape[0]

    for i in range(row):
        j = np.nonzero(edited_matr[i] == 1)[0][0]
        edited_matr = np.delete(edited_matr, j, 1)

    return edited_matr

In [ ]:
def get_lead_indexes(matrix):
    ref_matrix = ref(matrix)
    row = ref_matrix.shape[0]

    return [np.nonzero(ref_matrix[i] == 1)[0][0] for i in range(row)]

In [ ]:
def join_matrix(matrix):
    X = reduce_matrix(matrix)

    x_row = X.shape[0]
    x_column = X.shape[1]

    I = np.eye(x_column, dtype = int)

    index_x = 0
    index_i = 0

    result_matrix = np.zeros((x_row + x_column, x_column), dtype = int)

    lead_idxs = get_lead_indexes(matrix)
    for i in range (x_row + x_column):
        if(i in lead_idxs):
            result_matrix[i,:] = X[index_x,:]
            index_x += 1
        else:
            result_matrix[i,:] = I[index_i,:]
            index_i += 1

    return result_matrix

In [ ]:
def loop_sum_row(matrix):
    rref_matrix = rref(matrix)
    res = [np.zeros(rref_matrix.shape[1], dtype=int)]
    for row in rref_matrix:
        for i in range(len(res)):
            res.append((res[i] + row) % 2)

    s = np.array([tuple(i) for i in res]) # собираем в матрицу

    return np.unique(s, axis=0)

In [ ]:
def mult_by_g_sum_row(matrix):
    rref_matrix = rref(matrix)
    k = rref_matrix.shape[0]

    bins = [[int(j) for j in bin(i)[2:].zfill(k)] for i in range(2 ** k)]

    return np.array([np.dot(i, rref_matrix) % 2 for i in bins])

In [ ]:
def compare_sums(sum1, sum2):
    unique_elems = np.unique(np.concatenate((sum1, sum2), axis = 0), axis=0)
    return len(unique_elems) == len(sum1) == len(sum2)

In [ ]:
def mult_sum_by_H(sumN, H):
    return np.array([np.dot(i, H) % 2 for i in sumN])

In [ ]:
def calc_dist(matrix):
    min_d = len(matrix[0])
    for i in range(len(matrix[0])):
        for j in range(i + 1, len(matrix[0])):
            min_d = min(min_d, sum((matrix[i] + matrix[j]) % 2))
    return min_d, (min_d - 1)

In [ ]:
def make_error(sum1, sum2, H):
    d, t = calc_dist(sum2)

    row = sum2.shape[0]
    col = sum2.shape[1]

    flagNonDetected = True
    for i in range(row):
        if not (sum2[i] == np.zeros(col, dtype=int)).all() and flagNonDetected:
#             сначала ищем ошибку, которую можно заметить
            while True:
                detected_err_word = np.zeros(col, dtype=int)
                for _ in range(t):
                    idx = rnd.randint(0, col - 1)
                    detected_err_word[idx] = 1
                sum_res1 = (sum2[i] + detected_err_word) % 2
                res1 = np.dot(sum_res1, H) % 2

                if not (res1 == np.zeros(len(res1), dtype=int)).all():
#                     теперь которую не заметить
                    while True:
                        err_word = np.zeros(col, dtype=int)
                        for _ in range(t + 1):
                            idx = rnd.randint(0, col - 1)
                            err_word[idx] = 1
                        sum_res = (sum2[i] + err_word) % 2
                        res = np.dot(sum_res, H) % 2
                        if (res == np.zeros(len(res), dtype=int)).all():
                            print(f"v = {sum2[i]}\n")
                            print(f"e1 = {detected_err_word}")
                            print(f"v + e1 = {sum_res1}")
                            print(f"(v + e1)@H = {res1}\n")

                            print(f"e2 = {err_word}")
                            print(f"v + e2 = {sum_res}")
                            print(f"(v + e2)@H = {np.dot(sum_res, H) % 2}")
                            flagNonDetected = False
                            break
                    break


In [ ]:
matr = np.array([
    [1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
    [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
    [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
    [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
    [1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0],
    [1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0]
])

In [ ]:
ref(matr)

array([[1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]])

In [ ]:
rref(matr)

array([[1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]])

In [ ]:
reduce_matrix(matr)

array([[0, 1, 1, 1, 1, 0],
       [0, 0, 1, 0, 1, 1],
       [0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 1, 1]])

In [ ]:
H = join_matrix(matr)
H

array([[0, 1, 1, 1, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 1],
       [0, 0, 0, 1, 0, 1],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1]])

In [ ]:
sum1 = loop_sum_row(matr)
sum1

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1],
       [0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
       [1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1],
       [1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1],
       [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0],
       [1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1],
       [1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0],
       [1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 1, 1, 0, 1,

In [ ]:
sum2 = mult_by_g_sum_row(matr)
sum2

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1],
       [0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
       [1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1],
       [1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1],
       [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0],
       [1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1],
       [1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0],
       [1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 1, 1, 0, 1,

In [ ]:
compare_sums(sum1, sum2)

True

In [ ]:
mult_sum_by_H(sum1, H)

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int32)

In [ ]:
mult_sum_by_H(sum2, H)

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int32)

In [ ]:
d, t = calc_dist(sum2)
print("d =", d, "\nt =", t)

d = 2 
t = 1


In [ ]:
make_error(sum1, sum2, H)

v = [0 0 0 0 0 0 0 0 1 1 1]

e1 = [0 0 0 0 0 1 0 0 0 0 0]
v + e1 = [0 0 0 0 0 1 0 0 1 1 1]
(v + e1)@H = [0 0 1 0 0 0]

e2 = [0 0 0 0 0 0 1 0 1 0 0]
v + e2 = [0 0 0 0 0 0 1 0 0 1 1]
(v + e2)@H = [0 0 0 0 0 0]
